In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/Users/abohane/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
costs = pd.read_csv('data/cost_v1.csv')
dwell_time = pd.read_csv('data/dwell_time_v1.csv')
mall_traffic = pd.read_csv('data/mall_traffic.csv')
mall = pd.read_csv('data/mall_v1.csv')
people_flow = pd.read_csv('data/people_flow_v1.csv')
referenciel_traffic = pd.read_csv('data/referenciel_traffic_v1.csv')
store = pd.read_csv('data/store_v1.csv')

In [3]:
##Merging the dataframes in one

#Merging 'store', 'costs' and 'referenciel_traffic' on 'store_id' key.
df = pd.merge(referenciel_traffic, store, on='store_id', how='inner')
df = pd.merge(df, costs, on='store_id' , how='inner')

#Merging 'people_flow' and 'df' on 'block_id' key.
df = pd.merge(df, people_flow, on=['block_id', 'external_mall_id'], how='inner')

#Merging 'mall' and 'df' on 'mall_id' key.
df = pd.merge(df, mall, on='mall_id', how='inner')

#Merging 'mall_traffic', 'dwell_time' and 'df' on 'external_mall_id' key.
df = pd.merge(df, mall_traffic, on='external_mall_id', how='inner')
df = pd.merge(df, dwell_time, on=['external_mall_id', 'block_id'], how='inner')

In [4]:
df.shape

(3646, 39)

In [5]:
# 1. Traffic-Related Metrics

# Capture Rate: Store visits / total mall visits
df['capture_rate'] = (
    df['store_visits_monthly_average'] / df['visits_in_monthly_average']
)

# Conversion from Window Flow: Store visits / window flow
df['window_flow_conversion_rate'] = (
    df['store_visits_monthly_average'] / df['store_window_flow_monthly_average']
)

# Foot Traffic Share: Store visits / sum of visits for all stores (grouped by mall)
df['total_store_visits_in_mall'] = (
    df.groupby('mall_id')['store_visits_monthly_average']
      .transform('sum')
)
df['foot_traffic_share'] = (
    df['store_visits_monthly_average'] / df['total_store_visits_in_mall']
)

In [6]:
# 2. Revenue-Related Metrics

# Revenue per Store Visit: Store revenue / store visits
df['revenue_per_store_visit'] = (
    df['store_sales_r12m'] / df['store_total_visits']
)

# Revenue per Mall Visitor: Store revenue / total mall visits
df['revenue_per_mall_visitor'] = (
    df['store_sales_r12m'] / df['total_visits_in']
)

# Revenue per Square Foot: Store revenue / store area
df['revenue_per_sq_ft'] = (
    df['store_sales_r12m'] / df['store_area']
)

# Conversion Rate from Window Flow to Revenue (two-step)
df['conv_window_flow_to_visits'] = (
    df['store_total_visits'] / df['store_total_window_flow']
)
df['revenue_per_visit'] = (
    df['store_sales_r12m'] / df['store_total_visits']
)
df['revenue_per_window_flow'] = (
    df['conv_window_flow_to_visits'] * df['revenue_per_visit']
)

In [7]:
# 3. Dwell-Time-Related Metrics

# Average Dwell Time in Store vs. Mall Average
df['dwell_time_vs_mall'] = (
    df['avg_dwell_time'] - df['average_dwell_time']
)

# Dwell Time per Square Foot
# First approximate total dwell time: avg_dwell_time * total visits
df['total_dwell_time'] = (
    df['avg_dwell_time'] * df['store_total_visits']
)
df['dwell_time_per_sq_ft'] = (
    df['total_dwell_time'] / df['store_area']
)

In [8]:

# 4. Efficiency & Operational Metrics

# Store Visit Efficiency: revenue / (visits * avg dwell time)
df['store_visit_efficiency'] = (
    df['store_sales_r12m'] / (df['store_total_visits'] * df['avg_dwell_time'])
)

In [9]:
df.head()

,store_id,block_id,external_mall_id,store_name,mall_id,store_year_opened,store_year_closed,store_branch_level_1,store_branch_level_2,store_branch_level_3,...,revenue_per_store_visit,revenue_per_mall_visitor,revenue_per_sq_ft,conv_window_flow_to_visits,revenue_per_visit,revenue_per_window_flow,dwell_time_vs_mall,total_dwell_time,dwell_time_per_sq_ft,store_visit_efficiency
0,1845.0,7413,30,store_287,6,08/07/2022,NaN,Sport,Sport Clothes & Equipment,Sport clothes & Outdoor,...,7.209998,0.340369,4330.159529,0.101978,7.209998,0.735262,-539.786418,34301419.85,36725.288919,0.117907
1,1271.0,4680,10,store_15,4,11/10/2017,NaN,Sport,Sport Clothes & Equipment,Sport clothes & Outdoor,...,2.882957,0.083217,4780.515152,0.070055,2.882957,0.201964,-219.150274,27170397.45,91482.819697,0.052256
2,1451.0,4605,10,store_781,4,03/03/2023,NaN,Sport,Sport Clothes & Equipment,Sport clothes & Outdoor,...,NaN,NaN,NaN,0.013357,NaN,NaN,-209.652293,7231683.60,35978.525373,NaN
3,12114.0,11596,32,store_1440,68,13/09/2011,NaN,Sport,Sport Clothes & Equipment,Sport general store,...,6.495148,0.172503,2241.426591,0.081869,6.495148,0.531752,-650.808563,28064004.34,20588.216901,0.108869
4,1886.0,7444,30,store_24,6,23/10/2023,NaN,Sport,Sport Clothes & Equipment,Sport clothes & Outdoor,...,9.266581,0.062812,3554.042857,0.014650,9.266581,0.135757,-203.561187,4925143.30,23453.063333,0.151539


In [10]:
traffic_metrics = [
    'capture_rate',
    'window_flow_conversion_rate',
    'foot_traffic_share'
]

revenue_metrics = [
    'revenue_per_store_visit',
    'revenue_per_mall_visitor',
    'revenue_per_sq_ft',
    'conv_window_flow_to_visits',
    'revenue_per_visit',
    'revenue_per_window_flow'
]

dwell_metrics = [
    'dwell_time_vs_mall',
    'dwell_time_per_sq_ft'
]

efficiency_metrics = [
    'store_visit_efficiency'
]

# Helper function to min-max scale a Series
def min_max_scaler(series):
    min_val = series.min()
    max_val = series.max()
    # To avoid division by zero if all values are the same
    if min_val == max_val:
        return pd.Series([1.0]*len(series), index=series.index)
    return (series - min_val) / (max_val - min_val)

# Normalize all metrics in each group
for col in (traffic_metrics + revenue_metrics + dwell_metrics + efficiency_metrics):
    df[f"{col}_norm"] = min_max_scaler(df[col])

# Compute sub-scores (mean of normalized metrics within each category)
df['traffic_score'] = df[[f"{m}_norm" for m in traffic_metrics]].mean(axis=1)
df['revenue_score'] = df[[f"{m}_norm" for m in revenue_metrics]].mean(axis=1)
df['dwell_time_score'] = df[[f"{m}_norm" for m in dwell_metrics]].mean(axis=1)
df['efficiency_score'] = df[[f"{m}_norm" for m in efficiency_metrics]].mean(axis=1)

# Compute overall score as the mean of sub-scores (adjust weighting if desired)
df['overall_score'] = df[['traffic_score','revenue_score','dwell_time_score','efficiency_score']].mean(axis=1)

# Optionally, create ranks (1 = best/highest score)
df['traffic_rank'] = df['traffic_score'].rank(ascending=False, method='dense')
df['revenue_rank'] = df['revenue_score'].rank(ascending=False, method='dense')
df['dwell_time_rank'] = df['dwell_time_score'].rank(ascending=False, method='dense')
df['efficiency_rank'] = df['efficiency_score'].rank(ascending=False, method='dense')
df['overall_rank'] = df['overall_score'].rank(ascending=False, method='dense')

# Sort the DataFrame by overall_score descending (highest score first)
df.sort_values(by='overall_score', ascending=False, inplace=True)

In [11]:
df['mall_id'].nunique()

21

In [12]:
# Code to classify which `store_branch_level_1` have the best average scores.
# We'll group by `store_branch_level_1` and compute the mean scores,
# then rank them to see which branch level performs best on average.


# 1) Group by store_branch_level_1, compute the average of key scores
branch_scores = (
    df
    .groupby('store_branch_level_1')
    .agg({
        'traffic_score': 'mean',
        'revenue_score': 'mean',
        'dwell_time_score': 'mean',
        'efficiency_score': 'mean',
        'overall_score': 'mean'
    })
    .reset_index()
    .rename(columns={
        'traffic_score': 'avg_traffic_score',
        'revenue_score': 'avg_revenue_score',
        'dwell_time_score': 'avg_dwell_time_score',
        'efficiency_score': 'avg_efficiency_score',
        'overall_score': 'avg_overall_score'
    })
)

# 2) Sort by the average overall score (descending: best = highest avg score)
branch_scores.sort_values(by='avg_overall_score', ascending=False, inplace=True)

# 3) (Optional) Create a rank column to indicate position (1 = best)
branch_scores['avg_overall_rank'] = branch_scores['avg_overall_score'].rank(
    method='dense',
    ascending=False
)

# Preview results
branch_scores.head(10)  # Show the top 10 store_branch_level_1 categories


,store_branch_level_1,avg_traffic_score,avg_revenue_score,avg_dwell_time_score,avg_efficiency_score,avg_overall_score,avg_overall_rank
3,Department Stores,0.188624,0.088836,0.404995,0.0,0.191106,1.0
8,Food Stores & Mass Merchandise,0.121986,0.086302,0.478016,0.0,0.188924,2.0
16,Sport,0.047700,0.027108,0.533844,0.0,0.164483,3.0
11,Home,0.033709,0.026917,0.517498,0.0,0.156198,4.0
9,Gifts,0.034673,0.019053,0.521947,0.0,0.155516,5.0
5,Fashion apparel,0.032209,0.020901,0.513978,0.0,0.155395,6.0
1,Bags & Footwear & Accessories,0.022329,0.014933,0.517142,0.0,0.150438,7.0
2,Culture & Media & Technology,0.029811,0.016264,0.495981,0.0,0.145403,8.0
12,Jewellery,0.012115,0.007640,0.502051,0.0,0.141169,9.0
10,Health & Beauty,0.021965,0.011706,0.472493,0.0,0.136110,10.0


In [13]:
# 1) Aggregate number of stores per mall_id, category
mall_category_counts = (
    df
    .groupby(['mall_id', 'store_branch_level_1'])
    .agg(num_stores_in_category=('store_id', 'count'),
         total_area_in_category=('store_area', 'sum'),
         total_sales_in_category=('store_sales_r12m', 'sum'))
    .reset_index()
)

In [14]:
mall_category_pivot = mall_category_counts.pivot(
    index='mall_id',
    columns='store_branch_level_1',
    values='num_stores_in_category'
).fillna(0)

mall_category_pivot.columns = [
    f"count_{col}" for col in mall_category_pivot.columns
]

mall_category_pivot.head()


,count_Auto,count_Bags & Footwear & Accessories,count_Culture & Media & Technology,count_Department Stores,count_Entertainment,count_Fashion apparel,count_Fitness,count_Food & Beverage Services,count_Food Stores & Mass Merchandise,count_Gifts,count_Health & Beauty,count_Home,count_Jewellery,count_Luxury,count_Other,count_Services,count_Sport
mall_id,,,,,,,,,,,,,,,,,
0,0.0,15.0,13.0,0.0,3.0,55.0,0.0,40.0,5.0,8.0,33.0,7.0,13.0,0.0,5.0,13.0,7.0
1,1.0,15.0,19.0,0.0,5.0,57.0,0.0,42.0,2.0,9.0,34.0,14.0,12.0,0.0,4.0,7.0,9.0
4,0.0,22.0,12.0,0.0,4.0,92.0,0.0,30.0,2.0,9.0,33.0,14.0,16.0,0.0,4.0,6.0,13.0
5,0.0,24.0,21.0,0.0,2.0,76.0,0.0,19.0,3.0,6.0,30.0,8.0,15.0,2.0,3.0,7.0,9.0
6,0.0,17.0,17.0,0.0,2.0,58.0,1.0,16.0,1.0,4.0,29.0,5.0,18.0,0.0,2.0,5.0,14.0


In [15]:
# Suppose we extract the mall-level columns from df:
mall_info = (
    df.groupby('mall_id')
      .agg({
          'total_visits_in': 'max',
          'total_visits_out': 'max',
          'total_mall_area': 'max',
          'average_dwell_time': 'max'
       })
      .reset_index()
)

# Merge with the pivoted category counts
mall_features = mall_category_pivot.merge(mall_info, on='mall_id', how='left')

mall_features.head()


,mall_id,count_Auto,count_Bags & Footwear & Accessories,count_Culture & Media & Technology,count_Department Stores,count_Entertainment,count_Fashion apparel,count_Fitness,count_Food & Beverage Services,count_Food Stores & Mass Merchandise,...,count_Home,count_Jewellery,count_Luxury,count_Other,count_Services,count_Sport,total_visits_in,total_visits_out,total_mall_area,average_dwell_time
0,0,0.0,15.0,13.0,0.0,3.0,55.0,0.0,40.0,5.0,...,7.0,13.0,0.0,5.0,13.0,7.0,12082021,11577497,108440.8,4083.717198
1,1,1.0,15.0,19.0,0.0,5.0,57.0,0.0,42.0,2.0,...,14.0,12.0,0.0,4.0,7.0,9.0,11852501,11402733,120306.9,3988.332570
2,4,0.0,22.0,12.0,0.0,4.0,92.0,0.0,30.0,2.0,...,14.0,16.0,0.0,4.0,6.0,13.0,17061664,17271390,91831.5,4033.282611
3,5,0.0,24.0,21.0,0.0,2.0,76.0,0.0,19.0,3.0,...,8.0,15.0,2.0,3.0,7.0,9.0,9001134,8743147,65330.2,4469.666905
4,6,0.0,17.0,17.0,0.0,2.0,58.0,1.0,16.0,1.0,...,5.0,18.0,0.0,2.0,5.0,14.0,11882295,11138257,77325.5,4388.318917


In [16]:
mall_features.shape

(21, 22)

In [17]:
X = mall_features.drop(columns=['total_visits_in', 'mall_id'])
mall_ids = pd.DataFrame(mall_features['mall_id'])
y = mall_features['total_visits_in']

In [18]:
mall_ids.head(21)


mall_ids['mall_name'] = df[df['mall_id'].isin(mall_ids['mall_id'])]['mall_name'].unique()

In [19]:
mall_ids.drop(columns=['mall_id'], inplace=True)
mall_ids.to_csv('mall_ids.csv', index=False)

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

# 1. Split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
mse_rf = ((y_pred_rf - y_test) ** 2).mean()
mape_value = mean_absolute_percentage_error(y_test, y_pred_rf) * 100
print("MAPE: {:.2f}%".format(mape_value))
print("Random Forest MSE:", mse_rf)
print("Random Forest R^2 Score:", rf.score(X_test, y_test))

MAPE: 9.68%
Random Forest MSE: 2863310158018.2476
Random Forest R^2 Score: 0.9531507891914689


In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_absolute_percentage_error

# 1. Split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

# ------------------------------------------------------
# 2. Ridge Regression
# ------------------------------------------------------
ridge = Ridge(alpha=1.0, random_state=42)  # alpha is the regularization strength
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)

# Metrics for Ridge
mape_ridge = mean_absolute_percentage_error(y_test, y_pred_ridge) * 100
mse_ridge = np.mean((y_test - y_pred_ridge) ** 2)
r2_ridge = ridge.score(X_test, y_test)

print("Ridge Regression:")
print(f"  MAPE: {mape_ridge:.2f}%")
print(f"  MSE:  {mse_ridge:.2f}")
print(f"  R²:   {r2_ridge:.4f}")
print("")

# ------------------------------------------------------
# 3. Lasso Regression
# ------------------------------------------------------
lasso = Lasso(alpha=0.1, random_state=42)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

# Metrics for Lasso
mape_lasso = mean_absolute_percentage_error(y_test, y_pred_lasso) * 100
mse_lasso = np.mean((y_test - y_pred_lasso) ** 2)
r2_lasso = lasso.score(X_test, y_test)

print("Lasso Regression:")
print(f"  MAPE: {mape_lasso:.2f}%")
print(f"  MSE:  {mse_lasso:.2f}")
print(f"  R²:   {r2_lasso:.4f}")


Ridge Regression:
  MAPE: 15.69%
  MSE:  9719603955084.58
  R²:   0.8410

Lasso Regression:
  MAPE: 93.47%
  MSE:  230881255764834.16
  R²:   -2.7777


In [23]:
def predict_scenario_before_after(model, mall_data, scenario_changes, feature_cols):
    """
    model: RandomForestRegressor
    mall_data: a single row from the mall_features DataFrame (excluding 'mall_id' and the target)
    scenario_changes: dict of changes in categories
    feature_cols: the list of columns used in your model
    """
    # 1) Create a copy of the base row (before scenario change)
    base_scenario = mall_data.copy()

    # 2) Create a copy for the new scenario
    new_scenario = mall_data.copy()

    # 3) Apply the deltas in the scenario_changes dictionary
    for cat_col, delta in scenario_changes.items():
        # If cat_col wasn't in the row, default to 0
        new_scenario[cat_col] = new_scenario.get(cat_col, 0) + delta

    # 4) Convert each scenario to a 1-row DataFrame and reindex to match the model's feature columns
    base_df = pd.DataFrame([base_scenario]).reindex(columns=feature_cols, fill_value=0)
    new_df = pd.DataFrame([new_scenario]).reindex(columns=feature_cols, fill_value=0)

    # 5) Predict using the model
    visits_before = model.predict(base_df)[0]
    visits_after = model.predict(new_df)[0]

    return visits_before, visits_after

# Example usage:

scenario_changes = {
    'count_Apparel': 3,
    'count_Electronics': -2
}

visits_before, visits_after = predict_scenario_before_after(
    rf,
    X_train.iloc[1,:],
    scenario_changes,
    X.columns
)

print("Visits before the scenario change:", visits_before)
print("Visits after the scenario change:",  visits_after)



Visits before the scenario change: 22153249.43
Visits after the scenario change: 22153249.43


In [24]:
X.to_csv('data/mall_features.csv', index=False)

In [25]:

drastic_scenario_changes = {
    'count_Fashion apparel': +4,
    'count_Food & Beverage Services': +8,
    'count_Fitness': -5
}

visits_before, visits_after = predict_scenario_before_after(
    rf,
    X.iloc[0],
    drastic_scenario_changes,
    X.columns
)

print("Visits before the scenario change:", visits_before)
print("Visits after the scenario change:", visits_after)
print("Difference in visits:", visits_after - visits_before)


Visits before the scenario change: 14383168.5
Visits after the scenario change: 14400442.87
Difference in visits: 17274.36999999918


In [26]:
import pickle

# Suppose 'ridge' is your trained model
with open("trained_rf.pkl", "wb") as f:
    pickle.dump(rf, f)